In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os

In [ ]:
os.chdir("/AIS-AI/")

In [ ]:
!pip install -r requirements.txt

In [ ]:
import tensorflow as tf
import dataset
#from utils import Dataset, plotImages, plotWrongImages, plotHistory, inputdata
#from utils import Dataset, plotImages, plotWrongImages, plotHistory
from models import EfficientCapsNet

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
# some parameters
model_name = 'AIS'

In [ ]:
!python dataset.py

In [ ]:
!unzip "/Data/AISdata.zip" -d "/content"

In [ ]:
from dataset import Dataset

# 1.0 Import the Dataset

In [ ]:
dataset = Dataset(model_name, config_path='/Efficient-CapsNet-main_AIS/config.json')

In [ ]:
dataset.class_names

In [ ]:
dataset.X_train.shape

## 1.1 Visualize imported dataset

In [ ]:
n_images = 2 # number of images to be plotted
plotImages(dataset.X_test[:n_images,...,0], dataset.y_test[:n_images], n_images, dataset.class_names)

# 2.0 Load the Model

In [ ]:
model_train = EfficientCapsNet(model_name, mode='train', verbose=True)

(None, 74, 49, 32)
(None, 15, 10, 64)
Model: "Efficient_CapsNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150, 100, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 74, 49, 32)        416       
_________________________________________________________________
batch_normalization (BatchNo (None, 74, 49, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 24, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 36, 24, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 11, 64)        18496     
_________________________________________________________________
batch_norma

# 3.0 Train the Model

In [ ]:
dataset_train, dataset_val = dataset.get_tf_data() 

In [ ]:
history = model_train.train(dataset, initial_epoch=0)

In [ ]:
# some parameters
model_name = 'AIS' 
custom_path = '/model/AIS_model.h5' # if you've trained a new model, insert here the full graph weights path

In [ ]:
model_test = EfficientCapsNet(model_name, mode='test', verbose=True, custom_path=custom_path)

model_test.load_graph_weights() # load graph weights (bin folder)

(None, 74, 49, 32)
(None, 15, 10, 64)
Model: "Efficient_CapsNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150, 100, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 74, 49, 32)        416       
_________________________________________________________________
batch_normalization (BatchNo (None, 74, 49, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 24, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 36, 24, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 11, 64)        18496     
_________________________________________________________________
batch_norma

In [ ]:
model_test.evaluate(dataset.X_test, dataset.y_test)

------------------------------AIS Evaluation------------------------------
Test acc: 0.7636363636363637
Test error [%]: 23.6364%
NÃ‚Â° misclassified images: 25 out of 110


In [ ]:
y_pred = model_test.predict(dataset.X_test)[0]

for i in range(0,110):
  print (i, y_pred[i])

In [ ]:
Y_pred_0 = [y[1] for y in y_pred]

In [ ]:
Y_pred_0

In [ ]:
import numpy as np

In [ ]:
import sklearn.metrics as metrics

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
labels=np.argmax((dataset.y_test), axis=1)
pre=np.argmax(model_test.predict(dataset.X_test)[0], axis=1)

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(labels, Y_pred_0)

In [ ]:
roc_auc = metrics.auc(fpr, tpr)

In [ ]:
import matplotlib.pyplot as plt
plt.title('ROC curve for AIS progression predictor')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
dataset.class_names

In [ ]:
np.set_printoptions(threshold=5000)

In [ ]:
print(np.argmax(model_test.predict(dataset.X_test)[0], axis=1))

In [ ]:
print (np.argmax((dataset.y_test), axis=1))

In [ ]:

from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
 
guess = np.argmax(model_test.predict(dataset.X_test)[0], axis=1)
fact = np.argmax((dataset.y_test), axis=1)
classes = list(set(fact))
classes.sort()
confusion = confusion_matrix(guess, fact)
plt.imshow(confusion, cmap=plt.cm.Blues)
indices = range(len(confusion))
plt.xticks(indices, classes)
plt.yticks(indices, classes)
plt.colorbar()
plt.xlabel('Predict')
plt.ylabel('True')
for first_index in range(len(confusion)):
    for second_index in range(len(confusion[first_index])):
        plt.text(first_index, second_index, confusion[first_index][second_index])

plt.show()

In [ ]:
for i in range(0,110):
  if np.argmax(dataset.y_test[i]) != np.argmax(model_test.predict(dataset.X_test)[0][i]):
    print (i)

In [ ]:
img=model_test.predict(dataset.X_test)[1]

In [ ]:
from utils import plotImages, plotWrongImages, plotHistory

In [ ]:
plotHistory(history)